##### import libs

In [1]:
import numpy as np
import pandas as pd
import implicit
import matplotlib.pyplot as plt
from tqdm import tqdm
from pathlib import Path
from scipy.sparse import csr_matrix
from sklearn.model_selection import train_test_split

In [2]:
from utils import evaluate, load_data

##### read data

In [3]:
user_item_data, user_meta_data, item_meta_data, test_pairs_data = load_data()

##### split data

In [4]:
ui_train, ui_val = train_test_split(user_item_data,
                                    test_size=0.15,
                                    random_state=42,
                                    shuffle=False)

In [5]:
u_train = ui_train.user_id
i_train = ui_train.item_id
likes_train = ui_train.like
dislikes_train = ui_train.dislike

u_val = ui_val.user_id
i_val = ui_val.item_id
likes_val = ui_val.like
dislikes_val = ui_val.dislike

In [6]:
sparse_train = csr_matrix((likes_train, (u_train, i_train)))
sparse_val = csr_matrix((likes_val, (u_val, i_val)))

##### als model

In [15]:
sparse_train = implicit.nearest_neighbours.tfidf_weight(sparse_train)

In [ ]:
model = implicit.als.AlternatingLeastSquares(factors=128,
                                             random_state=42)
model.fit(sparse_train)

##### evaluation

In [9]:
model = model.to_cpu()

/home/sokolov/workspace/vk_recsys/vk_env/lib/python3.10/site-packages/implicit/cpu/als.py:95: RuntimeWarning: OpenBLAS is configured to use 16 threads. It is highly recommended to disable its internal threadpool by setting the environment variable 'OPENBLAS_NUM_THREADS=1' or by calling 'threadpoolctl.threadpool_limits(1, "blas")'. Having OpenBLAS use a threadpool can lead to severe performance issues here.
  check_blas_config()


In [10]:
als_score = (model.user_factors[ui_val['user_id']] * 
             model.item_factors[ui_val['item_id']]).sum(axis=1)

In [ ]:
evaluate(user_id=ui_val.user_id.values,
         target=likes_val.values,
         score=als_score)

100%|██████████| 181721/181721 [00:49<00:00, 3666.21it/s]


0.5885649014823264

In [12]:
np.save("dumps/als_val_score.npy", als_score)

##### submission

In [ ]:
test_als_score = (model.user_factors[test_pairs_data['user_id']] * 
                  model.item_factors[test_pairs_data['item_id']]).sum(axis=1)

In [ ]:
test_pairs_data["predict"] = test_als_score
test_pairs_data.to_csv("./submission.csv",index=False)